In [1]:
import PIL
import pandas as pd
import numpy as np
import tensorflow as tf
import datasets
import os
import sys
from dotenv import load_dotenv
from sklearn.utils import class_weight
from sklearn.datasets import load_files
import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import wandb

c:\D\Internship\Adamata\trash-classification\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#this code only can run after import the wandb
from wandb.integration.keras import WandbModelCheckpoint, WandbMetricsLogger

In [3]:
ds = datasets.load_dataset("garythung/trashnet", split=datasets.ReadInstruction("train", from_=2527, to=5054, unit="abs"))

In [4]:
#load path on .env
load_dotenv()

True

In [ ]:
#Check .env availability
try:
    from dotenv import load_dotenv
except ImportError:
    print("python-dotenv is not installed. Please install it by running:")
    print("pip install python-dotenv")
    sys.exit(1)

def check_env_file():
    """
    Attempt to load environment variables from a .env file.
    Returns True if .env was found and successfully loaded,
    otherwise returns False.
    """
    # load_dotenv() returns True if at least one environment variable
    # was set, or if the .env file was found. Otherwise, returns False.
    return load_dotenv()

if check_env_file():
    print(".env file found and environment variables are loaded.")
else:
    print("No .env file found or environment variables were not set.")

# Check if specific variables are set
data_url = os.getenv("DATA_URL")
wandb_api_key = os.getenv("WANDB_API_KEY")
dataset_dict = os.getenv("DATASET_DICT")
artifact_dir = os.getenv("ARTIFACT_DIR")

if not data_url:
    print("WARNING: DATA_URL not set.")
    sys.exit(1)    
if not wandb_api_key:
    print("WARNING: WANDB_API not set.")
    sys.exit(1)
if not dataset_dict:
    print("WARNING: DATASET_DICT not set.")
    sys.exit(1)
if not artifact_dir:
    print("WARNING: ARTIFACT_DIR not set.")
    sys.exit(1)


## Data Preparation

In [5]:
#create metadata
job_type = "train_model"
config = {
    "optimizer": "adam",
    "loss": "categorical_crossentropy",
    "metrics": ["acc"],
    "epochs": 100,
    "validation_split": 0.1,
}

#initialization wandb for create the run session
run = wandb.init(
    project="trashnet-classification",
    job_type=job_type,
    config=config,
)

#access the latest dataset version with artifact then dowload
version = "latest"
name = "{}:{}".format("{}_dataset".format("trashnet"), version)
artifact = run.use_artifact(artifact_or_name=name)

artifact_dir = artifact.download()




wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: agungadipurwa (agungadipurwa_). Use `wandb login --relogin` to force relogin


wandb:   2528 of 2528 files downloaded.  


In [6]:
artifact_data=os.path.join(os.getcwd(),os.getenv("ARTIFACT_DIR"))

In [7]:
#get the config of metadata model versioning 
input_shape = (300, 300, 3)
loss = "categorical_crossentropy"
optimizer = run.config["optimizer"]
metrics = ["accuracy"]
epochs = run.config["epochs"]
validation_split = run.config["validation_split"]

In [11]:
#Generate tensorflow/keras dataset format from downloades artifacts dataset version 

#Generate train dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  artifact_data,
  validation_split=validation_split, #split by 90:10
  subset="training",
  seed=123, #make shuffle dataset from next generate
  image_size=(300, 300),
  batch_size=32,
  label_mode="categorical") #make sure label_mode on categorical so the target shape will (None, num_classes)

#Generate validation/test dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  artifact_data,
  validation_split=validation_split,#split by 90:10
  subset="validation",
  seed=123, #make shuffle dataset from next generate
  image_size=(300, 300),
  batch_size=32,
  label_mode="categorical")#make sure label_mode on categorical so the target shape will (None, num_classes)


Found 2527 files belonging to 6 classes.
Using 2275 files for training.
Found 2527 files belonging to 6 classes.
Using 252 files for validation.


In [10]:
n_classes = len(train_ds.class_names) #count the number of labels

In [12]:
def CNN_Model(input_shape,classes):
    #Generate manual augmentation layer without generator
    data_augmentation = tf.keras.Sequential([
        layers.RandomRotation(0.4),
        layers.RandomTranslation(0.2, 0.2),
        # layers.RandomZoom(0.1),
        layers.RandomFlip("horizontal"),
        layers.Rescaling(1./255)
    ])
    X_Input = layers.Input(input_shape)
    X = data_augmentation(X_Input) #augmented data before convulation

    #Convolution blocks
    X = layers.Conv2D(32,(3,3), padding='same',activation='relu')(X)
    X = layers.MaxPooling2D(pool_size=2)(X)

    X = layers.Conv2D(64,(3,3), padding='same',activation='relu')(X)
    X =layers.MaxPooling2D(pool_size=2)(X)

    X = layers.Conv2D(32,(3,3), padding='same',activation='relu')(X)
    X = layers.MaxPooling2D(pool_size=2)(X)

    #Classification layers
    X = layers.Flatten()(X)

    X = layers.Dense(64,activation='relu')(X)

    X = layers.Dropout(0.2)(X)
    X = layers.Dense(32,activation='relu')(X)

    X = layers.Dropout(0.2)(X)
    X = layers.Dense(classes,activation='softmax')(X)

    model = Model(inputs=X_Input,outputs=X, name="CNN")
    return model


In [13]:
#build the architecure model
model = CNN_Model(input_shape, n_classes) 

In [14]:
#compile the model with the optimizer, loss, and metric config
model.compile(optimizer=optimizer, loss=loss, metrics=metrics) 

In [15]:
#show the model architecture
model.summary()

Model: "CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 300, 300, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 300, 300, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 300, 300, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 150, 150, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 150, 150, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 75, 75, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43808)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,803,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,843,910 (10.85 MB)

 Trainable params: 2,843,910 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
#Create a custom callback Class for stoping traing base of the threshold
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs=None):
        #will stop the traning if validation accuracy and train accuracy higher than 90% or 99%
        if logs.get('val_accuracy') >= 0.90 or logs.get('accuracy') >= 0.99:
            self.model.stop_training = True

In [22]:
#Create list of callback method
callbacks_list = [
    #model checkpoint for local tracking
    ModelCheckpoint(
        filepath="ckpt/cnn_v2.weights.h5",
        monitor=metrics[0],
        mode="max",
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
        save_freq="epoch"),

    #model log for every epoch
    WandbMetricsLogger(log_freq="epoch"),

    #model checkpoint for tracking on Wandb
    WandbModelCheckpoint(
        filepath="ckpt/cnn_v2.weights.h5",
        monitor=metrics[0],
        mode="max",
        save_weights_only=True,
        save_best_only=True,
        save_freq="epoch"),

    #model stopping when reach the threshold
    CustomCallback()

    ##additional callback for experiment
    # ReduceLROnPlateau(
    #     monitor=metrics[0],       # Monitor Validation Loss Metric
    #     factor=0.5,               # Reduce learning rate around 50%
    #     patience=2,               # Reduce Learning Rate if didn't improvement while 5 epoch
    #     verbose=1,                # Showing Log
    #     min_lr=1e-6),               # Learning rate minimum

    # EarlyStopping(
    #     monitor="val_accuracy",
    #     mode="max",
    #     verbose=1,
    #     baseline=0.95,
    #     restore_best_weights=True)

    ]

Back to the Exploratory Data Analysis (EDA), that some problem with imbalanced data. The code below will try to deal with imbalanced data with balancing the class weigth

In [133]:
#Deals with Imbalanced Data

#Store all label
y_train = np.array(ds["label"])

#Compute the class weigth data dan store in array
#Idea of balanced weight is greated the weight of minority class
#Balanced formula: number of label(y_train)/number of the class

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

In [23]:
#build the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks_list,
    steps_per_epoch=len(train_ds),
    validation_steps=len(val_ds),
    # class_weight=class_weights, #include the class weight
    verbose=1)

Epoch 1/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 957ms/step - accuracy: 0.2368 - loss: 1.7258
Epoch 1: accuracy improved from -inf to 0.22769, saving model to ckpt/cnn_v2.weights.h5
72/72 ━━━━━━━━━━━━━━━━━━━━ 71s 998ms/step - accuracy: 0.2367 - loss: 1.7258 - val_accuracy: 0.2024 - val_loss: 1.6568
Epoch 2/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 903ms/step - accuracy: 0.2753 - loss: 1.6651
Epoch 2: accuracy improved from 0.22769 to 0.31341, saving model to ckpt/cnn_v2.weights.h5
72/72 ━━━━━━━━━━━━━━━━━━━━ 67s 933ms/step - accuracy: 0.2759 - loss: 1.6644 - val_accuracy: 0.3254 - val_loss: 2.1299
Epoch 3/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2852 - loss: 1.6856
Epoch 3: accuracy improved from 0.31341 to 0.32967, saving model to ckpt/cnn_v2.weights.h5
72/72 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.2858 - loss: 1.6845 - val_accuracy: 0.4286 - val_loss: 1.4552
Epoch 4/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4138 - loss: 1.4771
Epoch 4: accuracy improved from 0.32

In [24]:
#Saving model
path = "./ckpt/cnn_v2.weights.h5" #save modle on local
registered_model_name = "cnn_trashnet_v2" #push model for tacking versioning or experiment to Wandb

run.link_model(path=path, registered_model_name=registered_model_name)
run.finish() #close the running session of Wandb

epoch/accuracy,▁▂▂▂▄▄▄▅▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██
epoch/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,██▇▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
epoch/val_accuracy,▁▄▄▄▅▇▆▇▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇▇███████▇█
epoch/val_loss,▆█▄▄▄▄▃▃▃▃▃▂▂▂▄▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁
epoch/accuracy,0.74066
epoch/epoch,99
epoch/learning_rate,0.001
epoch/loss,0.69952
epoch/val_accuracy,0.77778
